In [232]:
import numpy as np

hydration_after_p = 0.8
hydration_before_p = 0.045
#Wet Weight(g) after pretreatment
#2% 1h, 12% 1h, 2% 3h 12% 3h
w_60c= np.array([7.8302,4.0316,5.45,4.48]) 
w_80c= np.array([6.4508, 5.181, 5.929, 3.9756]) 
w_100c = np.array([5.6054, 4.1634, 5.9361, 3.9318]) 
w_120c = np.array([4.8918, 3.3319, 5.1648, 2.9348]) 

#Dry Weight(g) after pretreatment
dry_w_60c = w_60c * (1-hydration_after_p)
dry_w_80c = w_80c * (1-hydration_after_p)
dry_w_100c = w_100c * (1-hydration_after_p)
dry_w_120c = w_120c * (1-hydration_after_p)

avg_lost_w_60c = dry_w_60c/2
avg_lost_w_80c = dry_w_80c/2
avg_lost_w_100c = dry_w_100c/2
avg_lost_w_120c = dry_w_120c/2

dry_w_60c, dry_w_80c, dry_w_100c, dry_w_120c

(array([1.56604, 0.80632, 1.09   , 0.896  ]),
 array([1.29016, 1.0362 , 1.1858 , 0.79512]),
 array([1.12108, 0.83268, 1.18722, 0.78636]),
 array([0.97836, 0.66638, 1.03296, 0.58696]))

In [233]:
def calculate_dry_weight(w, hydration_after_p):
    return w * (1 - hydration_after_p)

def calculate_hplc(hplc, k, m, dilution_factor=5):
    return np.array([dilution_factor*(value-m)/k for value in hplc])

def calculate_mass(c, volume_H):
    return c * volume_H

def calculate_max(values, dry_masses):
    return [values[i]*value for i, value in enumerate(dry_masses)]

def calculate_yield(m, max_m):
    return np.divide(m, max_m)

def calculate_average(pairs):
    return np.mean(pairs, axis=1)

def reshape_into_pairs(array):
    return array.reshape(-1, 2)

def format(input_array):
    # Multiply array by 100
    multiplied_array = input_array * 100

    # Convert to string
    str_array = np.array(multiplied_array, dtype=str)

    # Extract the first three characters
    extracted_array = np.array([x[:3] for x in str_array])
    
    return extracted_array


In [234]:
def preprocess_data(G_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_pretreatment):

    pairs_G_C = reshape_into_pairs(G_samples_hplc)
    pairs_XMG_C = reshape_into_pairs(XMG_samples_hplc)

    avgs_G_C = calculate_average(pairs_G_C)
    avgs_XMG_C = calculate_average(pairs_XMG_C)
  
    c_G = calculate_hplc(avgs_G_C, k=315718, m=4623)
    print(f"c_G: {c_G}")
    c_XMG = calculate_hplc(avgs_XMG_C, k=317400, m=10216)
    print(f"c_XMG: {c_XMG}")
    
    m_G_before = calculate_mass(c_G, volume)
    print(f"m_G: {m_G_before}")
    m_XMG_before = calculate_mass(c_XMG, volume)
    print(f"m_XMG: {m_XMG_before}")

    #Kompensera för att vi inte tog allt när vi gjorde hplc
    wet_mass_into_hplc = 2 #g
    moisture = 0.8
    dried_sample = wet_mass_into_hplc * (1-moisture)
    #print(f"dried sample: {dried_sample}")
    multiply_factor_over_pretreatment = dry_masses_after_p/dried_sample
    #print(f"multiply factor over pretreatment: {multiply_factor_over_pretreatment}")
    m_G = m_G_before * multiply_factor_over_pretreatment
    m_XMG = m_XMG_before * multiply_factor_over_pretreatment
    

    print(f"dry massses after p: {dry_masses_after_p}, dry_masses_before_pretreatment: {dry_masses_before_pretreatment}")
    #Over the process
    max_glucose_grams = glucose_yields[0]*dry_masses_before_pretreatment
    max_XMG_grams = xylmangal_values[0]*dry_masses_before_pretreatment
    max_monosacharices_grams = np.add(max_glucose_grams, max_XMG_grams)

    #print(f"max glucose grams: {max_glucose_grams}, max XMG grams: {max_XMG_grams}, max monosacharices grams: {max_monosacharices_grams}")

    yield_glucose_procent_process = calculate_yield(m_G, dry_masses_before_pretreatment)
    yield_XMG_procent_process = calculate_yield(m_XMG, dry_masses_before_pretreatment)
    yield_monosacharides_procent_process = np.add(yield_glucose_procent_process, yield_XMG_procent_process)

    #Over the pretreatment
    print(f"glucose_yields[1:]: {glucose_yields[1:]}, dry_masses_after_p: {dry_masses_after_p}")
    max_glucose_p_grams = calculate_max(glucose_yields[1:], dry_masses_after_p)
    #print(f"max glucose p grams: {max_glucose_p_grams}")
    max_XMG_p_grams = calculate_max(xylmangal_values[1:], dry_masses_after_p)

   
    max_monosacharices_p_grams = np.add(max_glucose_p_grams, max_XMG_p_grams)

    print(f"m_G: {m_G}, max_glucose_p_grams: {max_glucose_p_grams}")
    yield_glucose_procent_pretreatment = calculate_yield(m_G, max_glucose_p_grams)
    yield_XMG_procent_pretreatment = calculate_yield(m_XMG, max_XMG_p_grams)
    yield_monosacharides_procent_pretreatment = np.add(yield_glucose_procent_pretreatment, yield_XMG_procent_pretreatment)

    #Selectivity
    

    #Over the process
    selectivity_glucose_procent_process = np.divide(m_G_before, 0.4)
    selectivity_XMG_procent_process = np.divide(m_XMG_before, 0.4)
    

    results = {
        "Dry Milled(g)": dry_masses_before_pretreatment,
        "Dry after Pretreatment(g)": dry_masses_after_p,

        "HPLC: Glucose(g) ": m_G,
        "HPLC: XMG(g)": m_XMG,
        "Monosacharides(g)": np.add(m_G, m_XMG),

        "Selctivity Glucose over Hydrolysis": format(selectivity_glucose_procent_process),
        "Selctivity XMG over Hydrolysis": format(selectivity_XMG_procent_process),

        "NREL Yield Glucose": 39.54,
        "NREL Yield XMG": 25.24,
        "NREL Yield Monosacharides": 25.24+39.54,

        "Actual Yield Glucose over Process": format(yield_glucose_procent_process),
        "Actual Yield XMG over Process": format(yield_XMG_procent_process),
        "Actual Yield Monosacharides over Process": format(yield_monosacharides_procent_process),

        #"Max Yield Glucose over Pretreatment": glucose_yields[1:],
        #"Max Yield XMG over Pretreatment": xylmangal_values[1:],
        #"Max Yield Monosacharides over Pretreatment": glucose_yields[1:]+xylmangal_values[1:],
#
        #"Actual Yield Glucose over Hydrolysis": yield_glucose_procent_pretreatment,
        #"Actual Yield XMG over Hydrolysis": yield_XMG_procent_pretreatment,
        #"Actual Yield Monosacharides over Hydrolysis": yield_monosacharides_procent_pretreatment,
    }

    return results

In [235]:
#Varmast 1h, varmast 3h, kallast 1h, kallast 3h
#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h

#CHANGE THESE
milled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])
Glucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])
XMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])

#Constants
glucose_yields = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100
xylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100
volume = 0.02485
hydration_after_p = 0.8

dry_masses_before_p = milled_masses*0.955
average_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))
a = average_dry_masses_before_pretreatment
print(f"Average dry masses before pretreatment: {a}")

#CHANGE THIS 
dry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])

print(f"dryu masses after p: {dry_masses_after_p}")

Average dry masses before pretreatment: [1.999961   2.00258725 2.0008205  2.00096375]
dryu masses after p: [0.97834092 1.03429626 1.56668247 1.09052524]


In [236]:
data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, average_dry_masses_before_pretreatment)
import pandas as pd

index = ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
#df = pd.DataFrame(data)
#print(df)

c_G: [ 9.56662077 10.65774638  9.85976281  9.17624451]
c_XMG: [2.27159542 2.55920425 2.36562247 2.20246647]
m_G: [0.23773053 0.264845   0.24501511 0.22802968]
m_XMG: [0.05644915 0.06359623 0.05878572 0.05473129]
dry massses after p: [0.97834092 1.03429626 1.56668247 1.09052524], dry_masses_before_pretreatment: [1.999961   2.00258725 2.0008205  2.00096375]
glucose_yields[1:]: [0.883  0.6842 0.6511 0.5628], dry_masses_after_p: [0.97834092 1.03429626 1.56668247 1.09052524]
m_G: [0.58145376 0.68482048 0.95965218 0.6216803 ], max_glucose_p_grams: [0.8638750341083398, 0.707665503062496, 1.0200669548562005, 0.6137476071824999]
Filtered array [59.43263156 66.21124938 61.25377645]
Filtered array [14.11228655 15.89905639 14.69642959]
Filtered array [29.07325471 34.19678608 47.96293204]
Filtered array [ 6.90344833  8.21154465 11.50759829]
Filtered array [35.97670304 42.40833073 59.47053033]


In [237]:
# General data that is common to both dataframes
general_data = [
    "Dry Milled(g)",
    "Dry after Pretreatment(g)",
    "HPLC: Glucose(g) ",
    "HPLC: XMG(g)",
    "Monosacharides(g)",
    "Selctivity Glucose over Hydrolysis",
    "Selctivity XMG over Hydrolysis"
    
]

# Process specific data
process_data_keys = general_data + [
    "NREL Yield Glucose",
    "NREL Yield XMG",
    "NREL Yield Monosacharides",
    "Actual Yield Glucose over Process",
    "Actual Yield XMG over Process",
    "Actual Yield Monosacharides over Process"
]

# Pretreatment specific data
#pretreatment_data_keys = general_data + [
#    "Max Yield Glucose over Pretreatment",
#    "Max Yield XMG over Pretreatment",
#    "Max Yield Monosacharides over Pretreatment",
#    "Actual Yield Glucose over Hydrolysis",
#    "Actual Yield XMG over Hydrolysis",
#    "Actual Yield Monosacharides over Hydrolysis"
#]

# Create dataframes
process_data = {key: data[key] for key in process_data_keys}
#pretreatment_data = {key: data[key] for key in pretreatment_data_keys}

process_df = pd.DataFrame(process_data)
#pretreatment_df = pd.DataFrame(pretreatment_data)

# Assign index
#process_df.index = pretreatment_df.index = index
process_df.index = index

print('2% NaOH batch - Process Data')
print(process_df)



ValueError: All arrays must be of the same length

In [ ]:
print('\n2% NaOH batch - Pretreatment Data')
#print(pretreatment_df)


2% NaOH batch - Pretreatment Data
